<a href="https://colab.research.google.com/github/ko-ko-go/Data-flow-Credit-Scoring/blob/main/creditscoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# --- STEP 1: INSTALL & IMPORT ---
!pip install kagglehub pandas-gbq db-sqlite3 xgboost imbalanced-learn

import kagglehub
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import brier_score_loss, roc_auc_score, classification_report
from imblearn.over_sampling import SMOTE
from google.colab import auth

# --- STEP 2: LOAD & PREPARE DATA ---
print("📥 กำลังดาวน์โหลดข้อมูล...")
path = kagglehub.dataset_download("urstrulyvikas/lending-club-loan-data-analysis")
csv_file = [f for f in os.listdir(path) if f.endswith('.csv')][0]
full_path = os.path.join(path, csv_file)

df = pd.read_csv(full_path)
print("✅ โหลดข้อมูลเรียบร้อย!")

# จำลอง Database (SQLite)
conn = sqlite3.connect(':memory:')
df.to_sql('loan_data', conn, index=False)

# SQL Transformation
sql_query = """
SELECT
    *,
    CASE
        WHEN fico >= 750 THEN 'Excellent'
        WHEN fico >= 700 THEN 'Good'
        WHEN fico >= 660 THEN 'Fair'
        ELSE 'Poor'
    END AS fico_category,
    EXP("log.annual.inc") AS annual_income
FROM loan_data
"""
df_processed = pd.read_sql(sql_query, conn)

# ⚠️ แก้ไขจุดสำคัญ: เปลี่ยนชื่อคอลัมน์ทั้งหมด (ลบจุดออก) ตั้งแต่ตรงนี้เลย
df_processed.columns = df_processed.columns.str.replace('.', '_')
print("✅ เปลี่ยนชื่อคอลัมน์เรียบร้อย (เช่น not.fully.paid -> not_fully_paid)")

# สร้าง df_model (แปลง Category เป็นตัวเลข)
df_model = pd.get_dummies(df_processed, columns=['purpose', 'fico_category'], drop_first=True)

# --- STEP 3: ADVANCED MODELING (XGBoost + SMOTE) ---
print("⚙️ กำลังเริ่มกระบวนการ Modeling...")

X = df_model.drop(['not_fully_paid', 'annual_income', 'credit_policy'], axis=1)
y = df_model['not_fully_paid']

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# แก้ Imbalanced Data ด้วย SMOTE
print("⚖️ กำลังทำ SMOTE แก้ข้อมูลไม่สมดุล...")
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# สร้างและเทรนโมเดล XGBoost
print("🚀 กำลังเทรน XGBoost...")
xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    scale_pos_weight=(len(y_train[y_train==0]) / len(y_train[y_train==1])),
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Calibrate Probability
calibrated_model = CalibratedClassifierCV(xgb_model, method='isotonic', cv=3)
calibrated_model.fit(X_train_resampled, y_train_resampled)

# Evaluate
print("\n--- Model Performance ---")
y_prob = calibrated_model.predict_proba(X_test)[:, 1]
print(f"ROC AUC Score: {roc_auc_score(y_test, y_prob):.4f}")

# Apply Model to Full Data
# เตรียม X สำหรับ predict ทั้งหมด (ต้อง drop column ให้เหมือนตอน train)
X_all = df_model.drop(['not_fully_paid', 'annual_income', 'credit_policy'], axis=1)
df_processed['probability_of_default'] = calibrated_model.predict_proba(X_all)[:, 1]

# --- STEP 4: FIND OPTIMAL THRESHOLD (PROFIT MAXIMIZATION) ---
print("\n💰 กำลังคำนวณจุดตัดกำไรสูงสุด (Profit Matrix)...")

# สมมติฐานทางธุรกิจ (Business Assumptions)
# ในความเป็นจริงคุณอาจคำนวณจาก installment * term - loan_amount ก็ได้
# แต่ใช้ค่าคงที่เพื่อความง่ายในการ Simulation ถือว่าโอเคครับ
PROFIT_PER_GOOD_LOAN = 1000  # กำไรจากดอกเบี้ย
COST_OF_DEFAULT = -5000      # ขาดทุนจากเงินต้น

thresholds = np.linspace(0, 1, 101) # ไล่ระดับความเสี่ยงตั้งแต่ 0% ถึง 100%
results = []

actual_status = df_processed['not_fully_paid']
predicted_prob = df_processed['probability_of_default']

for t in thresholds:
    # สร้างการตัดสินใจ (1=ปล่อยกู้, 0=ไม่ปล่อย) ตาม threshold t
    # ถ้าความเสี่ยงต่ำกว่า t ให้ปล่อยกู้ (approve)
    approved_mask = predicted_prob < t

    # เลือกเฉพาะคนที่ได้รับอนุมัติ
    approved_loans_actual = actual_status[approved_mask]

    num_approved = len(approved_loans_actual)
    num_total = len(actual_status)

    if num_approved == 0:
        results.append({
            'threshold': t,
            'total_profit': 0,
            'approved_count': 0,
            'approval_rate': 0,
            'default_rate_in_portfolio': 0
        })
        continue

    # นับจำนวนหนี้ดี/หนี้เสีย ในพอร์ตที่อนุมัติ
    num_bad = approved_loans_actual.sum() # sum ของ 1 คือจำนวนคนเบี้ยวหนี้
    num_good = num_approved - num_bad

    # คำนวณกำไร
    total_profit = (num_good * PROFIT_PER_GOOD_LOAN) + (num_bad * COST_OF_DEFAULT)

    # คำนวณ Metrics สำหรับ Dashboard
    approval_rate = num_approved / num_total
    default_rate_in_portfolio = num_bad / num_approved # NPL Rate ของพอร์ตใหม่

    results.append({
        'threshold': t,
        'total_profit': total_profit,
        'approved_count': num_approved,
        'approval_rate': approval_rate,
        'default_rate_in_portfolio': default_rate_in_portfolio
    })

df_optimization = pd.DataFrame(results)

# หาจุดที่ดีที่สุด
best_scenario = df_optimization.loc[df_optimization['total_profit'].idxmax()]
optimal_threshold = best_scenario['threshold']

print(f"🏆 จุดตัดที่ดีที่สุด (Optimal Threshold): {optimal_threshold:.2f}")
print(f"💵 กำไรสูงสุดที่ทำได้: {best_scenario['total_profit']:,.0f} บาท")
print(f"📉 อัตราการอนุมัติ (Approval Rate): {best_scenario['approval_rate']*100:.2f}%")
print(f"⚠️ อัตราหนี้เสียในพอร์ต (Expected NPL): {best_scenario['default_rate_in_portfolio']*100:.2f}%")

# --- STEP 5: PREPARE FINAL DATA & UPLOAD ---

# 1. ตารางรายคน (Loan Level) - เพิ่มสถานะว่า "Approved" หรือไม่ ตามจุดตัดที่ดีที่สุด
df_processed['risk_grade'] = pd.qcut(df_processed['probability_of_default'], q=5, labels=['A', 'B', 'C', 'D', 'F'])
df_processed['client_id'] = "LOAN_" + df_processed.index.astype(str)
df_processed['suggested_action'] = np.where(df_processed['probability_of_default'] < optimal_threshold, 'Approve', 'Reject')


# Upload
auth.authenticate_user()
project_id = 'creditscoring-12345' # <--- ⚠️ แก้ Project ID ตรงนี้เป็นของคุณ
dataset_id = 'credit_data'
table_name = 'loan_risk_report'

print(f"☁️ กำลังอัปโหลดข้อมูลไปยัง BigQuery: {table_name}...")
df_processed.to_gbq(f"{project_id}.{dataset_id}.{table_name}", project_id=project_id, if_exists='replace')

# Upload ตาราง Profit Optimization (เผื่อทำกราฟ)
df_optimization.to_gbq(f"{project_id}.{dataset_id}.profit_optimization", project_id=project_id, if_exists='replace')

print("🎉 เสร็จสมบูรณ์! ข้อมูล 2 ตารางพร้อมใช้งานบน Looker Studio แล้ว")

📥 กำลังดาวน์โหลดข้อมูล...
Using Colab cache for faster access to the 'lending-club-loan-data-analysis' dataset.
✅ โหลดข้อมูลเรียบร้อย!
✅ เปลี่ยนชื่อคอลัมน์เรียบร้อย (เช่น not.fully.paid -> not_fully_paid)
⚙️ กำลังเริ่มกระบวนการ Modeling...
⚖️ กำลังทำ SMOTE แก้ข้อมูลไม่สมดุล...
🚀 กำลังเทรน XGBoost...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [09:18:28] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [09:18:28] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [09:18:29] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



--- Model Performance ---
ROC AUC Score: 0.6179

💰 กำลังคำนวณจุดตัดกำไรสูงสุด (Profit Matrix)...
🏆 จุดตัดที่ดีที่สุด (Optimal Threshold): 0.25
💵 กำไรสูงสุดที่ทำได้: 4,125,000 บาท
📉 อัตราการอนุมัติ (Approval Rate): 71.07%
⚠️ อัตราหนี้เสียในพอร์ต (Expected NPL): 6.57%
☁️ กำลังอัปโหลดข้อมูลไปยัง BigQuery: loan_risk_report...


/tmp/ipython-input-3190627530.py:174: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_processed.to_gbq(f"{project_id}.{dataset_id}.{table_name}", project_id=project_id, if_exists='replace')
100%|██████████| 1/1 [00:00<00:00, 12748.64it/s]
/tmp/ipython-input-3190627530.py:177: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_optimization.to_gbq(f"{project_id}.{dataset_id}.profit_optimization", project_id=project_id, if_exists='replace')
100%|██████████| 1/1 [00:00<00:00, 7584.64it/s]

🎉 เสร็จสมบูรณ์! ข้อมูล 2 ตารางพร้อมใช้งานบน Looker Studio แล้ว
